<a href="https://colab.research.google.com/github/emredeveloper/Text-to-Sql/blob/main/Text_to_Sql_Groq_DuckDB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [29]:
import os
from groq import Groq
import json
import duckdb
import sqlparse
import numpy as np
import pandas as pd
import time
from colorama import Fore, Back, Style, init
from tabulate import tabulate
from pyfiglet import Figlet

# Colorama'yı başlat
init(autoreset=True)

def chat_with_groq(client, prompt, model, response_format):
    completion = client.chat.completions.create(
        model=model,
        messages=[
            {
                "role": "user",
                "content": prompt
            }
        ],
        response_format=response_format
    )
    return completion.choices[0].message.content

def execute_duckdb_query(query):
    original_cwd = os.getcwd()
    os.chdir('data')
    try:
        conn = duckdb.connect(database=':memory:', read_only=False)
        query_result = conn.execute(query).fetchdf().reset_index(drop=True)
    finally:
        os.chdir(original_cwd)
    return query_result

def get_summarization(client, user_question, df, model):
    prompt = '''
    A user asked the following question pertaining to local database tables:

    {user_question}

    To answer the question, a dataframe was returned:

    Dataframe:
    {df}

    In a few sentences, summarize the data in the table as it pertains to the original user question. Avoid qualifiers like "based on the data" and do not comment on the structure or metadata of the table itself
    '''.format(user_question=user_question, df=df)

    return chat_with_groq(client, prompt, model, None)

def print_fancy_title():
    f = Figlet(font='slant')
    print(Fore.CYAN + f.renderText('DuckDB Query Generator'))

def print_menu():
    print(Fore.YELLOW + "\n=== Menu ===")
    print(Fore.WHITE + "1. Ask a question")
    print(Fore.WHITE + "2. View query history")
    print(Fore.WHITE + "3. Exit")

def print_query_result(sql_query, results_df, summarization, execution_time):
    print(Fore.GREEN + "\n=== Query ===")
    print(Fore.WHITE + sqlparse.format(sql_query, reindent=True, keyword_case='upper'))

    print(Fore.GREEN + "\n=== Results ===")
    print(tabulate(results_df, headers='keys', tablefmt='pretty', showindex=False))

    print(Fore.GREEN + "\n=== Summary ===")
    print(Fore.WHITE + summarization)

    print(Fore.CYAN + f"\nExecution time: {execution_time:.2f} seconds")

def main():
    print_fancy_title()
    print(Fore.WHITE + "You can ask questions about the data in the 'employees.csv' and 'purchases.csv' files.")

    # Use the Llama3 70b model
    model = "llama3-70b-8192"

    # Get the Groq API key and create a Groq client
    groq_api_key = "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"
    client = Groq(api_key=groq_api_key)

    # Load the base prompt
    with open('prompts/base_prompt.txt', 'r') as file:
        base_prompt = file.read()

    query_history = []

    while True:
        print_menu()
        choice = input(Fore.YELLOW + "Enter your choice (1/2/3): " + Fore.WHITE)

        if choice == '1':
            user_question = input(Fore.YELLOW + "\nAsk a question: " + Fore.WHITE)

            try:
                full_prompt = base_prompt.format(user_question=user_question)

                print(Fore.CYAN + "\nGenerating SQL query...")
                start_time = time.time()
                llm_response = chat_with_groq(client, full_prompt, model, {"type": "json_object"})

                result_json = json.loads(llm_response)
                if 'sql' in result_json:
                    sql_query = result_json['sql']
                    print(Fore.CYAN + "Executing query...")
                    results_df = execute_duckdb_query(sql_query)

                    summarization = get_summarization(client, user_question, results_df, model)
                    execution_time = time.time() - start_time

                    print_query_result(sql_query, results_df, summarization, execution_time)

                    query_history.append({
                        'question': user_question,
                        'sql': sql_query,
                        'results': results_df,
                        'summary': summarization,
                        'time': execution_time
                    })
                elif 'error' in result_json:
                    print(Fore.RED + "ERROR: Could not generate valid SQL for this question")
                    print(Fore.RED + result_json['error'])

            except Exception as e:
                print(Fore.RED + f"An error occurred: {str(e)}")

        elif choice == '2':
            if not query_history:
                print(Fore.YELLOW + "\nNo queries in history.")
            else:
                for i, query in enumerate(query_history):
                    print(Fore.CYAN + f"\n=== Query {i+1} ===")
                    print(Fore.WHITE + f"Question: {query['question']}")
                    print(Fore.WHITE + f"SQL: {query['sql']}")
                    print(Fore.WHITE + "Results:")
                    print(tabulate(query['results'], headers='keys', tablefmt='pretty', showindex=False))
                    print(Fore.WHITE + f"Summary: {query['summary']}")
                    print(Fore.WHITE + f"Execution time: {query['time']:.2f} seconds")

        elif choice == '3':
            print(Fore.GREEN + "Thank you for using the DuckDB Query Generator. Goodbye!")
            break

        else:
            print(Fore.RED + "Invalid choice. Please try again.")

if __name__ == "__main__":
    main()

    ____             __   ____  ____     ____                       
   / __ \__  _______/ /__/ __ \/ __ )   / __ \__  _____  _______  __
  / / / / / / / ___/ //_/ / / / __  |  / / / / / / / _ \/ ___/ / / /
 / /_/ / /_/ / /__/ ,< / /_/ / /_/ /  / /_/ / /_/ /  __/ /  / /_/ / 
/_____/\__,_/\___/_/|_/_____/_____/   \___\_\__,_/\___/_/   \__, /  
                                                           /____/   
   ______                           __            
  / ____/__  ____  ___  _________ _/ /_____  _____
 / / __/ _ \/ __ \/ _ \/ ___/ __ `/ __/ __ \/ ___/
/ /_/ /  __/ / / /  __/ /  / /_/ / /_/ /_/ / /    
\____/\___/_/ /_/\___/_/   \__,_/\__/\____/_/     
                                                  

You can ask questions about the data in the 'employees.csv' and 'purchases.csv' files.

=== Menu ===
1. Ask a question
2. View query history
3. Exit
Enter your choice (1/2/3): 1

Ask a question: get 5 empoloyees name

Generating SQL query...
Executing query...

=== Query ===
SEL

KeyboardInterrupt: Interrupted by user

In [36]:
%%writefile app.py
import os
import streamlit as st
from groq import Groq
import json
import duckdb
import sqlparse
import pandas as pd
import time
from colorama import Fore, Back, Style, init
from tabulate import tabulate
from pyfiglet import Figlet

# Colorama'yı başlat
init(autoreset=True)

def chat_with_groq(client, prompt, model, response_format):
    completion = client.chat.completions.create(
        model=model,
        messages=[
            {
                "role": "user",
                "content": prompt
            }
        ],
        response_format=response_format
    )
    return completion.choices[0].message.content

def execute_duckdb_query(query):
    original_cwd = os.getcwd()
    os.chdir('data')
    try:
        conn = duckdb.connect(database=':memory:', read_only=False)
        query_result = conn.execute(query).fetchdf().reset_index(drop=True)
    finally:
        os.chdir(original_cwd)
    return query_result

def get_summarization(client, user_question, df, model):
    prompt = '''
    A user asked the following question pertaining to local database tables:

    {user_question}

    To answer the question, a dataframe was returned:

    Dataframe:
    {df}

    In a few sentences, summarize the data in the table as it pertains to the original user question. Avoid qualifiers like "based on the data" and do not comment on the structure or metadata of the table itself
    '''.format(user_question=user_question, df=df)

    return chat_with_groq(client, prompt, model, None)

def print_fancy_title():
    f = Figlet(font='slant')
    st.text(Fore.CYAN + f.renderText('DuckDB Query Generator'))

def print_menu():
    st.text(Fore.YELLOW + "\n=== Menu ===")
    st.text(Fore.WHITE + "1. Ask a question")
    st.text(Fore.WHITE + "2. View query history")
    st.text(Fore.WHITE + "3. Exit")

def print_query_result(sql_query, results_df, summarization, execution_time):
    st.text(Fore.GREEN + "\n=== Query ===")
    st.text(Fore.WHITE + sqlparse.format(sql_query, reindent=True, keyword_case='upper'))

    st.text(Fore.GREEN + "\n=== Results ===")
    st.dataframe(results_df)

    st.text(Fore.GREEN + "\n=== Summary ===")
    st.text(Fore.WHITE + summarization)

    st.text(Fore.CYAN + f"\nExecution time: {execution_time:.2f} seconds")

def main():
    print_fancy_title()
    st.text(Fore.WHITE + "You can ask questions about the data in the 'employees.csv' and 'purchases.csv' files.")

    # Use the Llama3 70b model
    model = "llama3-70b-8192"

    # Get the Groq API key (replace with your actual key)
    groq_api_key = "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"
    client = Groq(api_key=groq_api_key)

    # Load the base prompt
    with open('prompts/base_prompt.txt', 'r') as file:
        base_prompt = file.read()

    query_history = []

    menu_choice = st.selectbox("Choose an option:", ["Ask a question", "View query history", "Exit"])

    if menu_choice == "Ask a question":
        user_question = st.text_input("Ask a question:")

        if st.button("Submit"):
            try:
                full_prompt = base_prompt.format(user_question=user_question)

                st.text(Fore.CYAN + "\nGenerating SQL query...")
                start_time = time.time()
                llm_response = chat_with_groq(client, full_prompt, model, {"type": "json_object"})

                result_json = json.loads(llm_response)
                if 'sql' in result_json:
                    sql_query = result_json['sql']
                    st.text(Fore.CYAN + "Executing query...")
                    results_df = execute_duckdb_query(sql_query)

                    summarization = get_summarization(client, user_question, results_df, model)
                    execution_time = time.time() - start_time

                    print_query_result(sql_query, results_df, summarization, execution_time)

                    query_history.append({
                        'question': user_question,
                        'sql': sql_query,
                        'results': results_df,
                        'summary': summarization,
                        'time': execution_time
                    })
                elif 'error' in result_json:
                    st.text(Fore.RED + "ERROR: Could not generate valid SQL for this question")
                    st.text(Fore.RED + result_json['error'])

            except Exception as e:
                st.text(Fore.RED + f"An error occurred: {str(e)}")

    elif menu_choice == "View query history":
        if not query_history:
            st.text(Fore.YELLOW + "\nNo queries in history.")
        else:
            for i, query in enumerate(query_history):
                st.text(Fore.CYAN + f"\n=== Query {i+1} ===")
                st.text(Fore.WHITE + f"Question: {query['question']}")
                st.text(Fore.WHITE + f"SQL: {query['sql']}")
                st.text(Fore.WHITE + "Results:")
                st.dataframe(query['results'])
                st.text(Fore.WHITE + f"Summary: {query['summary']}")
                st.text(Fore.WHITE + f"Execution time: {query['time']:.2f} seconds")

    elif menu_choice == "Exit":
        st.text(Fore.GREEN + "Thank you for using the DuckDB Query Generator. Goodbye!")

if __name__ == "__main__":
    main()


Overwriting app.py


In [ ]:
!streamlit run /content/app.py &>/content/logs.txt & npx localtunnel --port 8501 & curl ipv4.icanhazip.com

35.226.71.26
npx: installed 22 in 2.183s
your url is: https://eleven-tigers-warn.loca.lt
